In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
import polars as pl
import pandas as pd
import preprocessing.feature_engineering as fe
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score
from sklearn.preprocessing import StandardScaler

In [6]:
RSEED = 42

In [3]:
df_selected = pl.read_parquet("./data/2022_data_selected.parquet")

In [ ]:

#df_st4000 = df_selected.filter((pl.col("model") == "ST4000DM000"))
st400_dict = fe.create_faildate_dict(df_st4000)
df_st4000 = fe.create_faildate(df_st4000, st400_dict)
df_st4000 = fe.create_target_classification(df_st4000)


#df_st4000.write_parquet("./data/2022_data_ST4000DM000.parquet")

In [7]:
df = pl.read_parquet("./data/2022_data_ST4000DM000.parquet")

filter out faulty data entries:

In [8]:
faulty_data = df.filter(pl.col('Time_till_failure') < 0 )['serial_number'].unique()
faulty_data = faulty_data.to_list()

In [9]:
df_filter = df.filter( ~ pl.col("serial_number").is_in(faulty_data))

In [10]:
print(df.shape)
print(df_filter.shape)

(6699353, 40)
(6698587, 40)


In [11]:
df_filter = df_filter.to_pandas()

In [13]:
df_basemodel = df_filter.loc[:,["smart_5_raw","smart_187_raw","smart_188_raw","smart_197_raw","smart_198_raw","failing_in14days"]]
df_basemodel

,smart_5_raw,smart_187_raw,smart_188_raw,smart_197_raw,smart_198_raw,failing_in14days
0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0
3,0.0,1.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...
6698582,0.0,16.0,1.0,248.0,248.0,0
6698583,0.0,0.0,0.0,0.0,0.0,0
6698584,0.0,0.0,0.0,0.0,0.0,0
6698585,0.0,0.0,0.0,0.0,0.0,0


In [18]:
df_basemodel['mean'] = df_basemodel.iloc[:, 1:5].mean(axis=1)
df_basemodel

,smart_5_raw,smart_187_raw,smart_188_raw,smart_197_raw,smart_198_raw,failing_in14days,mean
0,0.0,0.0,0.0,0.0,0.0,0,0.00
1,0.0,0.0,0.0,0.0,0.0,0,0.00
2,0.0,0.0,0.0,0.0,0.0,0,0.00
3,0.0,1.0,0.0,0.0,0.0,0,0.25
4,0.0,0.0,0.0,0.0,0.0,0,0.00
...,...,...,...,...,...,...,...
6698582,0.0,16.0,1.0,248.0,248.0,0,128.25
6698583,0.0,0.0,0.0,0.0,0.0,0,0.00
6698584,0.0,0.0,0.0,0.0,0.0,0,0.00
6698585,0.0,0.0,0.0,0.0,0.0,0,0.00


In [21]:
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
df_basemodel = df_basemodel.dropna()

In [14]:
df_basemodel.shape

(6698163, 6)

In [15]:
df_target = df_basemodel.pop("failing_in14days")

In [16]:
df_target

0          0
1          0
2          0
3          0
4          0
          ..
6698582    0
6698583    0
6698584    0
6698585    0
6698586    0
Name: failing_in14days, Length: 6698163, dtype: int32

In [17]:
X_train, X_test, y_train, y_test = train_test_split(df_basemodel, df_target, train_size= 0.7, random_state=42, stratify=df_target)

In [18]:
st_scaler = StandardScaler()

In [19]:
X_train = st_scaler.fit_transform(X_train)

In [20]:
X_test = st_scaler.transform(X_test)

In [21]:
basemodel = LogisticRegression()

In [22]:
basemodel.fit(X_train, y_train)

LogisticRegression()

In [23]:
y_pred = basemodel.predict(X_test)

In [24]:
train_pred = basemodel.predict(X_train)

In [25]:
accuracy_score(y_test,y_pred)

0.9986643104652071

In [26]:
matrix = confusion_matrix(y_test, y_pred)

In [27]:
recall_score(y_test, y_pred)

0.033086804203970414

In [28]:
matrix

array([[2006680,     200],
       [   2484,      85]])

In [29]:
train_matrix = confusion_matrix(y_train, train_pred)
train_matrix

array([[4682245,     476],
       [   5749,     244]])

: 